In [ ]:

!pip install transformers datasets torch


import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from datasets import load_dataset


model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

dataset = load_dataset("Ram20307/carbonetralpathway")

dataset_split = dataset['train'].train_test_split(test_size=0.1)  # 10% for validation
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']


def preprocess_function(examples):
    inputs = tokenizer(examples['input'], truncation=True, padding="max_length", max_length=512)
    outputs = tokenizer(examples['output'], truncation=True, padding="max_length", max_length=512)


    labels = outputs['input_ids']
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': labels,
        'labels_attention_mask': outputs['attention_mask']
    }

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./flan_t5_finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=1e-4,
    logging_dir="./logs",
    save_strategy="epoch",
    evaluation_strategy="epoch"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.700200,0.057459
2,0.061500,0.048568


Epoch,Training Loss,Validation Loss
1,0.700200,0.057459
2,0.061500,0.048568
3,0.051600,0.045856


TrainOutput(global_step=2025, training_loss=0.2149973287700135, metrics={'train_runtime': 2983.3575, 'train_samples_per_second': 2.715, 'train_steps_per_second': 0.679, 'total_flos': 5546534554828800.0, 'train_loss': 0.2149973287700135, 'epoch': 3.0})

In [ ]:
# Load the tokenizer and model
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
import os

print(os.listdir(model_path))


['checkpoint-675', 'checkpoint-1350', 'checkpoint-2025']


In [ ]:
from huggingface_hub import login

login(token="your token")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Assuming your checkpoint directories are named checkpoint-1, checkpoint-2, etc.
checkpoint_path = "./flan_t5_finetuned/checkpoint-2025"  # Adjust as necessary
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)



In [ ]:
input_text = "carbon neutral pathway suggestion for 70000 tonnes of co2e"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_length=150,   # Increase as needed
    num_beams=5,      # Optional: for better results
    early_stopping=True
)

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)


to address the carbon emissions of 70000 tonnes of co2e, it is recommended to plant approximately 3,227,409 trees over an area of 1786.00 hectares. this afforestation effort is expected to yield around 1,220,000 carbon credits. by implementing this strategy, you will significantly contribute to carbon offset initiatives through co2 sequestration, thereby advancing environmental sustainability goals.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from huggingface_hub import HfApi, HfFolder

# Load the model and tokenizer
checkpoint_path = "./flan_t5_finetuned/checkpoint-2025"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# Define repository information
model_name = "samplefinetunedsih"  # Replace with your desired model repository name
model_repo = f"Ram20307/{model_name}"  # Replace 'your-username' with your Hugging Face username

# Create repository
api = HfApi()
api.create_repo(repo_id=model_repo, repo_type="model")

# Push the model
model.push_to_hub(model_repo)
tokenizer.push_to_hub(model_repo)

print(f"Model and tokenizer pushed to {model_repo}")


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

Model and tokenizer pushed to Ram20307/samplefinetunedsih


In [ ]:
from transformers import pipeline


pipe = pipeline("text2text-generation", model="Ram20307/samplefinetunedsih")

input_text = "carbon neutral pathway suggestion for 70000 tonnes of co2e"


output = pipe(input_text, max_length=150, num_return_sequences=1, do_sample=True, temperature=0.7)


#print("Pipeline Output:", output[0]['generated_text'])


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("Ram20307/samplefinetunedsih")
model = AutoModelForSeq2SeqLM.from_pretrained("Ram20307/samplefinetunedsih")

inputs = tokenizer(input_text, return_tensors="pt")


outputs = model.generate(
    **inputs,
    max_length=150,  # Increase the max length
    do_sample=True,  # Enable sampling for varied outputs
    top_k=50,        # Use top-k sampling for diversity
    top_p=0.95,      # Use top-p sampling
    temperature=0.7  # Adjust temperature for creativity
)


generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Manual Model Output:", generated_text)


Manual Model Output: to address the carbon emissions of 70000 tonnes of co2e, it is recommended to plant approximately 3,045,409 trees over an area of 1783.00 hectares. this afforestation effort is expected to yield around 1,220,000 carbon credits. by implementing this strategy, you will significantly contribute to carbon offset initiatives through co2 sequestration, thereby advancing environmental sustainability goals.
